In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import shutil

# download data
from six.moves.urllib.request import urlopen

# set env log level to supress messages, unless an error
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# set tf log level
tf.logging.set_verbosity(tf.logging.INFO)

# Helper to make the output consistent
SEED = 42
def reset_graph(seed=SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()

# helper to create dirs if they don't already exist
def maybe_create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print("{} created".format(dir_path))
    else:
        print("{} already exists".format(dir_path))
        
def maybe_fetch_data(url, file, dir_path):
    maybe_create_dir(dir_path)
    if not os.path.exists(file):
        # download and write data
        raw = urlopen(url).read()
        with open(file, "wb") as f:
            f.write(raw)
        print(file, "path written")
    else:
        print("{} already exists. Please rm to download new data".format(file))
    

# Version information
print("Python: {}".format(sys.version_info[:]))
assert "1.4" <= tf.__version__, "TensorFlow r1.4 or later is needed"
print('TensorFlow: {}'.format(tf.__version__))

# Check if using GPU
if not tf.test.gpu_device_name():
    print('No GPU found')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Python: (3, 6, 5, 'final', 0)
TensorFlow: 1.8.0
Default GPU Device: /device:GPU:0


In [2]:
## Download data paths
ROOT_DATA = "../ROOT_DATA/"
DATA_DIR = os.path.join(ROOT_DATA, "IRIS")

IRIS_TRAINING_PATH = os.path.join(DATA_DIR, "iris_training.csv")
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST_PATH = os.path.join(DATA_DIR, "iris_test.csv")
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

maybe_fetch_data(IRIS_TRAINING_URL, IRIS_TRAINING_PATH, DATA_DIR)
maybe_fetch_data(IRIS_TEST_URL, IRIS_TEST_PATH, DATA_DIR)

../ROOT_DATA/IRIS already exists
../ROOT_DATA/IRIS/iris_training.csv already exists. Please rm to download new data
../ROOT_DATA/IRIS already exists
../ROOT_DATA/IRIS/iris_test.csv already exists. Please rm to download new data


In [3]:
def create_hyper_params(start_fresh=False):
    global maybe_create_dir
    data_params = {}
    data_params['n_epochs'] = 500
    data_params['batch_size'] = 32
    data_params['buffer_size'] = 128 # for shuffling
    data_params['init_lr'] = 1e-2
    
    # dataset information, this is known information that will be 
    # used for the features. Header usually includes this info..
    data_params['feature_names'] = [
        'SepalLength',
        'SepalWidth',
        'PetalLength',
        'PetalWidth']
    
    data_params['out_dir'] = "./iris_custom_est"
    
    # by default, the project will NOT start fresh
    if start_fresh:
        if os.path.exists(data_params['out_dir']):
            shutil.rmtree(data_params['out_dir'], ignore_errors = True)

    if not os.path.exists(data_params['out_dir']):
        os.makedirs(data_params['out_dir'])
        print("{} created".format(data_params['out_dir']))
    else:
        print("{} already exists".format(data_params['out_dir']))
    
    return data_params

In [4]:
# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API
def my_input_fn(file_path, predict=False):
    global hyp_params
    if hyp_params['feature_names']:
        feat_types = [[0.]]*len(hyp_params['feature_names']) + [[0]]
        # [[0.0], [0.0], [0.0], [0.0], [0]]
        # [feat1, feat2, feat3, feat4, label]
    else:
        pass
        # TODO: this could read the csv header
    
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, feat_types)
        label = parsed_line[-1]  # Last element is the label
        features = parsed_line[:-1]  # all but last element
        d = dict(zip(hyp_params['feature_names'], features)), label
        return d

    dataset = (tf.data.TextLineDataset(file_path)  # Read text file
        .skip(1)  # Skip header row
        .map(decode_csv)  # Decode each line using custom decode_csv
        .shuffle(hyp_params['buffer_size'])  # (1 == no operation)
              )
    if predict:
        dataset = (dataset.repeat(1))
    else:
        dataset = (dataset.repeat(hyp_params['n_epochs']))    # Repeats dataset this # times
        
    dataset = (dataset.batch(hyp_params['batch_size'])
        .prefetch(1)  # Make sure you always have 1 batch ready to serve
    )
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

In [5]:
def my_model_fn(
    features, # This is batch_features from input_fn
    labels,   # This is batch_labels from input_fn
    mode # instance of tf.estimator.ModeKeys
    ):    # hyper parameters
    
    global hyp_params

    if mode == tf.estimator.ModeKeys.PREDICT:
        tf.logging.info("my_model_fn: PREDICT, {}".format(mode))
    elif mode == tf.estimator.ModeKeys.EVAL:
        tf.logging.info("my_model_fn: EVAL, {}".format(mode))
    elif mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info("my_model_fn: TRAIN, {}".format(mode))

    # TODO: this needs to be customized
    if hyp_params['feature_names']:
        feature_columns = [
            tf.feature_column.numeric_column(hyp_params['feature_names'][0]),
            tf.feature_column.numeric_column(hyp_params['feature_names'][1]),
            tf.feature_column.numeric_column(hyp_params['feature_names'][2]),
            tf.feature_column.numeric_column(hyp_params['feature_names'][3])
        ]
    else:
        pass
        # TODO: this logic may have already been performed and
        # could read the csv header

    # TODO: core model architecture logic
    # Create the layer of input
    input_layer = tf.feature_column.input_layer(features, feature_columns)
    h1 = tf.layers.dense(inputs=input_layer, units=10, activation=tf.nn.selu)
    h2 = tf.layers.dense(inputs=h1, units=10, activation=tf.nn.selu)
    # TODO: this could be len(features)
    logits = tf.layers.dense(inputs=h2, units=3)

    # class_ids will be the model prediction for the class (Iris flower type)
    # The output node with the highest value is our prediction
    predictions = { 'class_ids': tf.argmax(input=logits, axis=1), 'in_data': tf.identity(input=input_layer) }

    # 1. Prediction mode, Return our prediction. no need to add other ops
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # --- Evaluation and Training mode
    # Calculate the loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    accuracy = tf.metrics.accuracy(labels, predictions['class_ids'])

    # 2. Evaluation mode
    # Return our loss (which is used to evaluate our model)
    # Set the TensorBoard scalar my_accurace to the accuracy
    # Obs: This function only sets value during mode == ModeKeys.EVAL
    # To set values during training, see tf.summary.scalar
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode,
            loss=loss,
            eval_metric_ops={'my_accuracy': accuracy})

    # If mode is not PREDICT nor EVAL, then we must be in TRAIN
    assert mode == tf.estimator.ModeKeys.TRAIN, "TRAIN is only ModeKey left"

    # 3. Training mode
    # Provide global step counter (used to count gradient updates)
    #optimizer = tf.train.AdagradOptimizer(0.05)
    optimizer = tf.train.AdamOptimizer(hyp_params['init_lr'])
    train_op = optimizer.minimize(
        loss,
        global_step=tf.train.get_global_step())

    # Set the TensorBoard scalar my_accuracy to the accuracy
    # Obs: This function only sets the value during mode == ModeKeys.TRAIN
    # To set values during evaluation, see eval_metrics_ops
    tf.summary.scalar('my_accuracy', accuracy[1])

    # Return training operations: loss and train_op
    return tf.estimator.EstimatorSpec(
        mode,
        loss=loss,
        train_op=train_op)

## Initialize hyper-params

In [6]:
hyp_params = create_hyper_params(start_fresh=True)

./iris_custom_est created


## Build estimator

In [7]:
tf.logging.info("START estimator construction")
classifier = tf.estimator.Estimator(
    model_fn=my_model_fn,
    model_dir=hyp_params['out_dir'])  # Path to where checkpoints etc are stored
tf.logging.info("... END estimator construction")

INFO:tensorflow:START estimator construction
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './iris_custom_est', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe2b3cc0b70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:... END estimator construction


## Train Estimator

In [8]:
tf.logging.info("START classifier.train")
classifier.train(
    input_fn=lambda: my_input_fn(IRIS_TRAINING_PATH))
tf.logging.info("... END classifier.train")

INFO:tensorflow:START classifier.train
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:my_model_fn: TRAIN, train
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./iris_custom_est/model.ckpt.
INFO:tensorflow:loss = 2.377738, step = 0
INFO:tensorflow:global_step/sec: 393.238
INFO:tensorflow:loss = 0.17761636, step = 100 (0.256 sec)
INFO:tensorflow:global_step/sec: 448.392
INFO:tensorflow:loss = 0.12993188, step = 200 (0.222 sec)
INFO:tensorflow:global_step/sec: 308.074
INFO:tensorflow:loss = 0.02929898, step = 300 (0.324 sec)
INFO:tensorflow:global_step/sec: 311.446
INFO:tensorflow:loss = 0.13593495, step = 400 (0.327 sec)
INFO:tensorflow:global_step/sec: 411.382
INFO:tensorflow:loss = 0.026356565, step = 500 (0.237 sec)
INFO:tensorflow:global_step/sec: 504.046
INFO:tensorflow:loss = 0.12

In [9]:
# Evaluate our model using the examples contained in FILE_TEST
# Return value will contain evaluation_metrics such as: loss & average_loss
tf.logging.info("START classifier.evaluate")
evaluate_result = classifier.evaluate(
    input_fn=lambda: my_input_fn(IRIS_TEST_PATH))
tf.logging.info("... END classifier.evaluate")
tf.logging.info("Evaluation results")
for key in evaluate_result:
    tf.logging.info("   {}, was: {}".format(key, evaluate_result[key]))

INFO:tensorflow:START classifier.evaluate
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:my_model_fn: EVAL, eval
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-06-17:05:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./iris_custom_est/model.ckpt-1875
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-06-17:05:12
INFO:tensorflow:Saving dict for global step 1875: global_step = 1875, loss = 0.062088244, my_accuracy = 0.96666664
INFO:tensorflow:... END classifier.evaluate
INFO:tensorflow:Evaluation results
INFO:tensorflow:   loss, was: 0.062088243663311005
INFO:tensorflow:   my_accuracy, was: 0.9666666388511658
INFO:tensorflow:   global_step, was: 1875


In [10]:
# Predict the type of some Iris flowers.
# predict the examples in FILE_TEST, repeat only once (predict=True).
predict_results = classifier.predict(
    input_fn=lambda: my_input_fn(IRIS_TEST_PATH, predict=True))
tf.logging.info("Prediction on test file")
for idx, prediction in enumerate(predict_results):
    # Will print the predicted class, i.e: 0, 1, or 2 if the prediction
    # is Iris Setosa, Vericolor, Virginica, respectively.    
    iris_type = prediction["class_ids"]  # Get the predicted class (index)
    if iris_type == 0:
        tf.logging.info("...I think: {}, is Iris Setosa".format(prediction['in_data']))
    elif iris_type == 1:
        tf.logging.info("...I think: {}, is Iris Versicolor".format(prediction['in_data']))
    else:
        tf.logging.info("...I think: {}, is Iris Virginica".format(prediction['in_data']))

INFO:tensorflow:Prediction on test file
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:my_model_fn: PREDICT, infer
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./iris_custom_est/model.ckpt-1875
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:...I think: [5.  2.  5.7 2.5], is Iris Virginica
INFO:tensorflow:...I think: [5.7 2.5 6.7 3.3], is Iris Virginica
INFO:tensorflow:...I think: [1.1 0.1 4.3 3. ], is Iris Setosa
INFO:tensorflow:...I think: [4.9 2.  5.6 2.8], is Iris Virginica
INFO:tensorflow:...I think: [4.5 1.5 5.6 3. ], is Iris Versicolor
INFO:tensorflow:...I think: [4.  1.3 5.5 2.5], is Iris Versicolor
INFO:tensorflow:...I think: [4.9 1.5 6.3 2.5], is Iris Virginica
INFO:tensorflow:...I think: [4.6 1.4 6.1 3. ], is Iris Versicolor
INFO:tensorflow:...I think: [4.3 1.3 6.2 2.9], is Iris Versicolor
INFO:tensorflow:...I think: [3.5 1.  5.7 2.6], is Iris Versic

In [11]:
prediction_input = [[5.9, 3.0, 4.2, 1.5],  # -> 1, Iris Versicolor
                    [6.9, 3.1, 5.4, 2.1],  # -> 2, Iris Virginica
                    [5.1, 3.3, 1.7, 0.5],  # -> 0, Iris Setosa
                    [5.7, 2.5, 6.7, 3.3]]  # -> 2, Iris Virginica

In [12]:
def new_input_fn():
    global hyp_params
    def decode(x):
        # TODO: this is hardcoded currently
        x = tf.split(x, 4)  # Need to split into our 4 features
        return dict(zip(hyp_params['feature_names'], x))  # To build a dict of them

    dataset = tf.data.Dataset.from_tensor_slices(prediction_input)
    dataset = dataset.map(decode)
    iterator = dataset.make_one_shot_iterator()
    next_feature_batch = iterator.get_next()
    return next_feature_batch, None  # In prediction, we have no labels

In [13]:
# Predict all our prediction_input
predict_results = classifier.predict(input_fn=new_input_fn)

In [14]:
tf.logging.info("Predictions on memory")
for idx, prediction in enumerate(predict_results):
    type = prediction["class_ids"]  # Get the predicted class (index)
    if type == 0:
        tf.logging.info("...I think: {}, is Iris Setosa".format(prediction_input[idx]))
    elif type == 1:
        tf.logging.info("...I think: {}, is Iris Versicolor".format(prediction_input[idx]))
    else:
        tf.logging.info("...I think: {}, is Iris Virginica".format(prediction_input[idx]))

INFO:tensorflow:Predictions on memory
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:my_model_fn: PREDICT, infer
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./iris_custom_est/model.ckpt-1875
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:...I think: [5.9, 3.0, 4.2, 1.5], is Iris Versicolor
INFO:tensorflow:...I think: [6.9, 3.1, 5.4, 2.1], is Iris Virginica
INFO:tensorflow:...I think: [5.1, 3.3, 1.7, 0.5], is Iris Setosa
INFO:tensorflow:...I think: [5.7, 2.5, 6.7, 3.3], is Iris Virginica


In [15]:
#%%bash
#tensorboard --logdir "./slow"
# bad idea since I would need to stop it also...

In [16]:
# can be used to wipe logs / TF board
#shutil.rmtree("./slow", ignore_errors = True) # start fresh each time